In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
encoder='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
Preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [4]:
bert_Preprocess_model=hub.KerasLayer(Preprocess)

In [5]:
bert_encoder=hub.KerasLayer(encoder)

In [6]:
df = pd.read_csv('spam.csv',encoding="ISO-8859-1")

In [7]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
df = pd.concat([df['v1'],df['v2']],axis=1)

In [9]:
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [10]:
df['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [11]:
df.dtypes

v1    object
v2    object
dtype: object

In [12]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [13]:
x_train,x_test,y_train,y_test = train_test_split(df['v2'],df['v1'],random_state=42,test_size=0.2,stratify=df['v1'])

In [14]:
len(x_train)

4457

In [15]:
len(x_test)

1115

In [16]:
len(y_train)

4457

In [17]:
len(y_test)

1115

In [18]:
y_train.value_counts()

ham     3859
spam     598
Name: v1, dtype: int64

In [19]:
l=LabelEncoder()
y = l.fit_transform(y_train)

In [20]:
Preprossed_text = bert_Preprocess_model(['i love python'])
output = bert_encoder(Preprossed_text)

In [21]:
output.keys()

dict_keys(['default', 'sequence_output', 'pooled_output', 'encoder_outputs'])

In [22]:
#bert layer
Input = tf.keras.layers.Input(shape=(),dtype=tf.string,name='text')
Preprossed_text = bert_Preprocess_model(Input)
outputs= bert_encoder(Preprossed_text)

#Neural Network layer
l = tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
l=tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)

#Model
model = tf.keras.Model(inputs=[Input],outputs=[l])

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [24]:
Metrics=[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='Precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam',
             metrics=Metrics,
             loss='binary_crossentropy')

In [30]:
model.fit(np.array(x_train[:100]),np.array(y[:100]),epochs=10)

Epoch 1/10
4/4 [==============================] - 59s 13s/step - loss: 0.4087 - accuracy: 0.8500 - Precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/10
4/4 [==============================] - 52s 12s/step - loss: 0.4501 - accuracy: 0.8500 - Precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/10
4/4 [==============================] - 47s 11s/step - loss: 0.4485 - accuracy: 0.8500 - Precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 46s 10s/step - loss: 0.4637 - accuracy: 0.8500 - Precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 46s 10s/step - loss: 0.4004 - accuracy: 0.8500 - Precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 6/10
4/4 [==============================] - 46s 11s/step - loss: 0.3753 - accuracy: 0.8500 - Precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 7/10
4/4 [==============================] - 46s 11s/step - loss: 0.3601 - accuracy: 0.8500 - Precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 